# Efficient API calls

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
#Load API Credentials

with open('/Users/vinosraj/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

#instantiate YelpAPI variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [3]:
#set our API call parameters

LOCATION = 'Minneapolis, MN'
TERM = 'Indian'

In [5]:
#JSON file name

JSON_FILE = f"Data/results_in_progress_indian.json"
JSON_FILE

'Data/results_in_progress_indian.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_indian.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results+1)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

78

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

4

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,_C2rFjlRsKsMawQp8WTjiA,the-hyderabad-indian-grill-minneapolis-minneap...,The Hyderabad Indian Grill - Minneapolis,https://s3-media1.fl.yelpcdn.com/bphoto/DNkQau...,False,https://www.yelp.com/biz/the-hyderabad-indian-...,36,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 45.064837, 'longitude': -93.2490113}","[delivery, pickup]","{'address1': '765 53rd Ave NE', 'address2': No...",+17636571047,(763) 657-1047,11506.028102,NaN
1,uETQZ6X9HAGPzozGgqjtiQ,gorkha-palace-minneapolis,Gorkha Palace,https://s3-media4.fl.yelpcdn.com/bphoto/NCJvv7...,False,https://www.yelp.com/biz/gorkha-palace-minneap...,444,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.5,"{'latitude': 44.989277, 'longitude': -93.255079}","[delivery, pickup]","{'address1': '23 4th St NE', 'address2': '', '...",+16128863451,(612) 886-3451,3829.046150,$$
2,zu4zzewGmBYg6z_3vG1xFQ,godavari-truly-south-indian-restaurant-eden-pr...,Godavari Truly South Indian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/t-al8g...,False,https://www.yelp.com/biz/godavari-truly-south-...,41,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 44.851644, 'longitude': -93.429161}","[delivery, pickup]","{'address1': '566 Prairie Center Dr', 'address...",+19529418242,(952) 941-8242,16726.755905,NaN
3,EETP-7tP_eQf7JsK6D3hig,india-palace-uptown-minneapolis-3,India Palace Uptown,https://s3-media2.fl.yelpcdn.com/bphoto/oa54UC...,False,https://www.yelp.com/biz/india-palace-uptown-m...,191,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 44.956095, 'longitude': -93.2966238}","[delivery, pickup]","{'address1': '2546 Hennepin Ave S', 'address2'...",+16123775000,(612) 377-5000,1178.257656,$$
4,E8wLXmYBg8Af4lEoX4K7dg,tamarind-indian-cuisine-edina,Tamarind Indian Cuisine,https://s3-media3.fl.yelpcdn.com/bphoto/aQnrOy...,False,https://www.yelp.com/biz/tamarind-indian-cuisi...,42,"[{'alias': 'indpak', 'title': 'Indian'}]",4.5,"{'latitude': 44.8700388, 'longitude': -93.3280...","[delivery, pickup]","{'address1': '3875 Gallagher Dr', 'address2': ...",+19526817735,(952) 681-7735,11034.895687,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
73,C4BpEUEUEYG8y-14tajfzw,foxy-falafel-saint-paul,Foxy Falafel,https://s3-media2.fl.yelpcdn.com/bphoto/L1IHV0...,False,https://www.yelp.com/biz/foxy-falafel-saint-pa...,141,"[{'alias': 'mideastern', 'title': 'Middle East...",3.5,"{'latitude': 44.965217590332, 'longitude': -93...",[delivery],"{'address1': '791 Raymond Ave', 'address2': No...",+16518882255,(651) 888-2255,7286.869290,$$
74,Ysv8SsJ40PzqBbbU0tRfHA,tea-house-plymouth,Tea House,https://s3-media4.fl.yelpcdn.com/bphoto/v3g4tk...,False,https://www.yelp.com/biz/tea-house-plymouth?ad...,211,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,"{'latitude': 44.9789437571851, 'longitude': -9...","[pickup, delivery]","{'address1': '88 Nathan Ln N', 'address2': '',...",+17635443422,(763) 544-3422,9296.767045,$$
75,jc8lDXcgN9_QZrZEHxvHwQ,skyway-masala-st-paul,Skyway Masala,https://s3-media4.fl.yelpcdn.com/bphoto/tXDqfE...,False,https://www.yelp.com/biz/skyway-masala-st-paul...,7,"[{'alias': 'halal', 'title': 'Halal'}, {'alias...",4.0,"{'latitude': 44.94750777413626, 'longitude': -...","[pickup, delivery]","{'address1': '444 Cedar St', 'address2': 'Ste ...",+13205669759,(320) 566-9759,15593.798510,NaN
76,Rd8NDXv5JMaIdbRd1OBeLA,gyro-house-rogers-3,Gyro House,https://s3-media2.fl.yelpcdn.com/bphoto/T4N_dl...,False,https://www.yelp.com/biz/gyro-house-rogers-3?a...,37,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.0,"{'latitude': 45.21156, 'longitude': -93.55105}","[pickup, delivery]","{'address1': '14165 James Rd', 'address2': 'St...",+17635951140,(763) 595-1140,34173.982247,NaN
77,kv7WFRPzN6KYhN-1cOFxBQ,indian-zyka-eagan,Indian Zyka,,False,https://www.yelp.com/biz/indian-zyka-eagan?adj...,1,"[{'alias': 'indpak', 'title': 'Indian'}]",1.0,"{'latitude': 44.81887435913086, 'longitude': -...",[],"{'address1': '', 'address2': '', 'address3': '...",,,15250.414739,$$


In [16]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

1

In [26]:
final_df.drop_duplicates(subset='id', keep='last', inplace=True)

In [21]:
final_df['id'].map(lambda x: type(x)== 'list')

0     False
1     False
2     False
3     False
4     False
      ...  
73    False
74    False
75    False
76    False
77    False
Name: id, Length: 78, dtype: bool

In [27]:
final_df.duplicated(subset='id').sum()

0

In [28]:
final_df.head(3)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,_C2rFjlRsKsMawQp8WTjiA,the-hyderabad-indian-grill-minneapolis-minneap...,The Hyderabad Indian Grill - Minneapolis,https://s3-media1.fl.yelpcdn.com/bphoto/DNkQau...,False,https://www.yelp.com/biz/the-hyderabad-indian-...,36,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 45.064837, 'longitude': -93.2490113}","[delivery, pickup]","{'address1': '765 53rd Ave NE', 'address2': No...",+17636571047,(763) 657-1047,11506.028102,NaN
1,uETQZ6X9HAGPzozGgqjtiQ,gorkha-palace-minneapolis,Gorkha Palace,https://s3-media4.fl.yelpcdn.com/bphoto/NCJvv7...,False,https://www.yelp.com/biz/gorkha-palace-minneap...,444,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.5,"{'latitude': 44.989277, 'longitude': -93.255079}","[delivery, pickup]","{'address1': '23 4th St NE', 'address2': '', '...",+16128863451,(612) 886-3451,3829.046150,$$
2,zu4zzewGmBYg6z_3vG1xFQ,godavari-truly-south-indian-restaurant-eden-pr...,Godavari Truly South Indian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/t-al8g...,False,https://www.yelp.com/biz/godavari-truly-south-...,41,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 44.851644, 'longitude': -93.429161}","[delivery, pickup]","{'address1': '566 Prairie Center Dr', 'address...",+19529418242,(952) 941-8242,16726.755905,NaN


In [29]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_in_progress_indian.csv.gz', compression='gzip',
                index=False)